In [ ]:
!/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip
!pip install -qU "sagemaker"
!pip install -qU "boto3"

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role
role = get_execution_role()

In [ ]:
!mkdir docker

In [ ]:
%%writefile docker/Dockerfile
FROM 173754725891.dkr.ecr.us-east-1.amazonaws.com/sagemaker-spark-processing:3.2-cpu-py39-v1.0
RUN pip3 install pandas delta-spark==2.0.1 tldextract spark-nlp==3.4.1 azure-cosmos==4.2.0 pycryptodome
ENV PYTHONUNBUFFERED=TRUE
ENTRYPOINT ["python3"]

In [ ]:
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.Session().region_name
tag = ':latest'
ecr_repository = 'sagemaker-processing-container-3'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)
region, ecr_repository, processing_repository_uri

In [ ]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com 
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin 173754725891.dkr.ecr.{region}.amazonaws.com 
!docker build -t $ecr_repository docker
#!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $processing_repository_uri
!docker push $processing_repository_uri

In [ ]:
from sagemaker.spark.processing import PySparkProcessor

In [ ]:
spark_processor = PySparkProcessor(
    base_job_name="spark-preprocessor",
    image_uri = processing_repository_uri,
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    max_runtime_in_seconds=1200,
)

In [ ]:
from sagemaker.processing import ProcessingInput
input_location = "s3://aws-ml-blog/artifacts/delta-lake-bring-your-own-container/delta-table/california-housing/"
spark_processor.run(
    submit_app="deltaprocess.py",
    submit_jars= ["jars/delta-core_2.12-2.0.1.jar","jars/delta-storage-2.0.1.jar"],
    arguments=["--s3_input_bucket", input_location]
)